# Data engineering sur les données de vote + données sociales-éco-démo pour générer une table sur laquelle nous créerons des modèles de prédiction

In [414]:
import pandas as pd
import numpy as np 
import seaborn as sns
import glob, os
path = "C:\\Users\\Mohamed\\MS BGD\\fil_rouge_lepoint\\Predictions éléctions\\PRESIDENTIELLES_1965-2012-csv\\" 
path_to_csv = "C:\\Users\\Mohamed\\git\\Elections\\Analyses\\Jointure des tables\\"

### Import et manipulation data de votes 1965 - 2012

In [415]:
dft1 = pd.DataFrame()
dft2 = pd.DataFrame()

partis_politique = {"BALLADUR (UDF)": "centre", 'BARBU (DIV)':"xgauche", 'BARRE (UDF)':"centre", 'BAYROU (UDF)': "centre",\
                   'BESANCENOT (LCR)' : "xgauche", 'BOUCHARDEAU (PSU)': "xgauche", 'BOUSSEL (MPT)': "xgauche", 'BOUTIN (UDF DISS.)' :"centre",\
                   'BOVE' : 'vert', "BUFFET (PC)": "xgauche", 'CHABAN-DELMAS (UDR)': "droite", "CHEMINADE (POE)": "droite", \
                   'CHEVENEMENT (P. REP)':"gauche", 'CHIRAC (RPR)' : "droite", 'CHIRAC (UMP)': "droite", 'CREPEAU (MRG)': "gauche",\
                   'DE GAULLE (UNR)':"droite", 'DE VILLIERS (MPF)': "droite", "DE VILLIERS MPF)":"droite", 'DEBRE (DVD)':"droite",\
                   'DEFFERRE (SFIO)':"gauche", 'DUCATEL (DIV)':"xgauche",\
                   'DUCLOS (PCF)': "xgauche", 'DUMONT (ECO)':"vert", 'Eva JOLY (EELV)':"vert", \
                   'François BAYROU (MODEM)':"centre", 'François HOLLANDE (PS)':"gauche", 'GARAUD (DVD)':"droite",\
                   "GISCARD D'ESTAING (RI)":"centre", "GISCARD D'ESTAING (UDF)":"centre", 'GLUCKSTEIN (PT)':"xgauche",\
                   'HERAUD (DIV)':"xgauche", 'HUE (PCF)':"xgauche", 'JOSPIN (PS)':"gauche", 'JUQUIN ( )':"xgauche",\
                   'Jacques CHEMINADE (SP)':"droite", 'Jean-Luc MELENCHON (FG)':"xgauche", 'KRIVINE (LCR)':"xgauche",\
                   'LAGUILLER (LO)':"xgauche", 'LAJOINIE (PCF)':"xgauche", 'LALONDE (ECO)':"vert", 'LE PEN (FN)':"xdroite",\
                   'LECANUET (MRP)':"centre", 'LEPAGE (CAP 21)':"vert", 'MADELIN (DL)':"droite", 'MAMERE (VERTS)':"vert",\
                   'MARCHAIS (PCF)':"xgauche", 'MARCILHACY (DVD)':"droite", 'MEGRET (MNR)':"xdroite",\
                   'MITTERRAND (CIR)':"gauche", 'MITTERRAND (PS)':"gauche", 'MULLER (MDSR)':"droite",\
                   'Marine LE PEN (FN)': "xdroite", 'NIHOUS (CNPT)':"droite", 'Nathalie ARTHAUD (LO)':"xgauche",\
                   'Nicolas DUPONT-AIGNAN (DLR)':"droite", 'Nicolas SARKOZY (UMP)':"droite", 'POHER (CD)':"centre",\
                   'POMPIDOU (UDR)':"droite", 'Philippe POUTOU (NPA)':"xgauche", 'RENOUVIN (NAR)':"xdroite",\
                   'ROCARD (PSU)': "gauche", 'ROYAL (PS)': "gauche", 'ROYER (DVD)':"droite", 'SAINT-JOSSE (CNPT)':"droite",\
                   'SARKOZY (UMP)':"droite", 'SCHIVARDI (PT)':"xgauche", 'SEBAG (DIV)':"xgauche", 'TAUBIRA (PRG)':"gauche",\
                   'TIXIER-VIGNANCOUR (EXD)':"xdroite", 'VOYNET (VERTS)':"vert", 'WAECHTER (V)':"vert"}

#on liste les partis uniques
liste_parties_uniques = np.unique(list(partis_politique.values()))
#on boucle sur tous les fichiers
for file in os.listdir(path):
    
    #On garde que les csv
    if file.endswith(".csv"):
        
        df = pd.DataFrame()
        df = pd.read_csv(path+file)
        df = df.rename(columns={"Code du département": "code", "Code département": "code",\
                                "Code dÃ©partement": "code",\
                                "Libellé du département": "département", "Code département.1":"département",
                               'Code de la circonscription': "circonscription"})

        for partie in liste_parties_uniques:
            df[partie] = 0

        for col in df.columns: 
            if col in partis_politique:
                df[partis_politique[col]]+=df[col]
        df.drop(list(partis_politique.keys()),inplace=True,axis=1,errors='ignore')
        df["Année"] = file[10:14]

        if "t1" in file and "commp" not in file:
            dft1 = dft1.append(df)
        
        if "t2" in file and "commp" not in file:
            dft2 = dft2.append(df)
            
            

Index(['code', 'département', 'circonscription', 'Inscrits', 'Votants',
       'Exprimés', 'Blancs et nuls', 'centre', 'droite', 'gauche', 'vert',
       'xdroite', 'xgauche', 'Année'],
      dtype='object')

In [361]:
#print([item for item in dft2.columns if item not in list(partis_politique.keys()) ])
#print(list(dft1.columns))
#print(list(dft2.columns))

dft1["Abstention"] = dft1["Inscrits"] - dft1["Votants"]
dft2["Abstention"] = dft2["Inscrits"] - dft2["Votants"]

liste_parties_uniques = list(liste_parties_uniques)
liste_parties_uniques.append("Abstention")
liste_parties_uniques.append("Blancs et nuls")

for col in liste_parties_uniques:
    dft1["taux_"+col] = dft1[col]*100/dft1["Exprimés"]   
    dft2["taux_"+col] = dft2[col]*100/dft2["Exprimés"]   

dft1.to_csv(path_to_csv+"base_resultats_elections_t1.csv", index = False)
dft1.to_csv(path_to_csv+"base_resultats_elections_t2.csv", index = False)

In [362]:
print(dft1.columns)

print(liste_parties_uniques)

Index(['code', 'département', 'circonscription', 'Inscrits', 'Votants',
       'Exprimés', 'Blancs et nuls', 'centre', 'droite', 'gauche', 'vert',
       'xdroite', 'xgauche', 'Année', 'Abstention', 'taux_centre',
       'taux_droite', 'taux_gauche', 'taux_vert', 'taux_xdroite',
       'taux_xgauche', 'taux_Abstention', 'taux_Blancs et nuls'],
      dtype='object')
['centre', 'droite', 'gauche', 'vert', 'xdroite', 'xgauche', 'Abstention', 'Blancs et nuls']


In [363]:
print(dft1)

    code                       département circonscription  Inscrits  Votants  \
0      1                               AIN               1     68818    54262   
1      1                               AIN               2     70186    57221   
2      1                               AIN               3     67492    55503   
3      2                             AISNE               1     52618    46064   
4      2                             AISNE               2     68770    59908   
5      2                             AISNE               3     48492    42604   
6      2                             AISNE               4     53732    47000   
7      2                             AISNE               5     67138    58058   
8      3                            ALLIER               1     55694    45789   
9      3                            ALLIER               2     64120    54742   
10     3                            ALLIER               3     57620    48003   
11     3                    

In [364]:
dic = {}
taux_votes = ["taux_"+item for item in ['centre', 'droite', 'gauche', 'vert', 'xdroite', 'xgauche', 'Abstention', 'Blancs et nuls']]
for col in dft1.columns:
    if col in taux_votes:
        dic[col] = np.mean
    elif col in ["Inscrits", "Votants", "Exprimés", "Blancs et nuls", "centre", "droite","gauche", "vert", "xdroite", "xgauche"]:
        dic[col] = np.sum
    
dft1 = dft1.groupby(["département", "code", "Année"]).agg(dic).reset_index()


### Création des variables "départ frontaliers", "départ outre mer", "départ corse"

In [365]:
depart_frontaliers = ["PAS-DE-CALAIS","NORD", "AISNE", "ARDENNES", "MEUSE",
                      "MEURTHE-ET-MOSELLE", "MOSELLE", "BAS-RHIN", "HAUT-RHIN","TERRITOIRE DE BELFORT", "DOUBS", "JURA", "AIN", "HAUTE-SAVOIE", \
                      "SAVOIE", "HAUTES-ALPES", "ALPES-DE-HAUTE-PROVENCE", "ALPES-MARITIMES", \
                      "PYRENEES-ATLANTIQUES", "HAUTES-PYRENEES", "HAUTE-GARONNE",\
                      "ARIEGE", "PYRENEES-ORIENTALES", "AUDE", "HERAULT", "GARD",\
                     "BOUCHES-DU-RHONE", "VAR"]
depart_OM = ["GUADELOUPE", "GUYANE", "MARTINIQUE", "LA REUNION", "MAYOTTE"]
depart_corse = ["CORSE", "HAUTE-CORSE", "CORSE-DU-SUD"]

dft1["depart_frontalier"] = [1 if dep in depart_frontaliers else 0 for dep in dft1["département"]]
dft1["depart_OM"] = [1 if dep in depart_OM else 0 for dep in dft1["département"]]
dft1["depart_CORSE"] = [1 if dep in depart_corse else 0 for dep in dft1["département"]]

print(dft1[["depart_frontalier", "département", "depart_OM"]].head(1))
print(len(dft1["Année"].unique()))

   depart_frontalier département  depart_OM
0                  1         AIN          0
9


In [366]:
dft1.code = dft1.code.map(lambda x : "0"+str(x) if len(str(x))==1 else str(x))

## Merge table démographie

In [367]:
df_demo = pd.read_csv("data_civil_pop.csv")
df_demo = df_demo.drop("Unnamed: 0", axis=1)
df_demo["time_period"] = df_demo["time_period"] + 1

In [368]:
df_demo["time_period"].unique()
dft1["Année"] = dft1["Année"].map(lambda x : float(x))

In [369]:
df_merged = dft1.merge(df_demo, how='left', left_on=["code", "Année"], right_on=["dept_nb", "time_period"])

In [370]:
df_merged.head()
df_merged = df_merged.dropna(axis=0)

## Merge table emploi

In [371]:
print(df_merged["Année"].unique())

[ 1981.  1988.  1995.  2002.  2007.  2012.]


In [372]:
df_emploi = pd.read_csv("data_emploi.csv")
df_emploi = df_emploi.drop("Unnamed: 0", axis=1)

df_emploi["time_period"] = df_emploi["time_period"] + 1

df_emploi["var_chomage_annee"] = np.nan

for i in range(1, df_emploi.shape[0]): 
    try:
        df_emploi.iloc[i,7] = df_emploi.iloc[i, 6] - df_emploi.iloc[i, 3]
    except:
        continue
df_emploi["taux_chomage"] = (df_emploi["Q1_rate"] + df_emploi["Q2_rate"] + df_emploi["Q3_rate"] + df_emploi["Q4_rate"])/4

df_emploi["time_period"] = df_emploi["time_period"].map(lambda x : 1981 if x==1983 else x)


In [373]:
df_merged2 = df_merged.merge(df_emploi, how='left', left_on=["code", "Année"], right_on=["dept_nb", "time_period"])

In [374]:
df_merged2 = df_merged2.drop(["dept_nb_x", "dept_lib_x", "time_period_x", "time_period_y", "dept_lib_y", "dept_nb_y", \
                             'Exprimés', 'Blancs et nuls', 'centre', 'xdroite', "Votants", 'gauche', 'droite', 'vert', \
                             'Inscrits', 'xgauche'], axis=1)

In [375]:
df_merged2.to_csv("la_base.csv")
dft11 = df_merged2


### Zones de force

In [376]:
elections_par_annees = dft1.groupby("Année")[taux_votes].mean()
for taux in taux_votes:
    dft11[taux+"_sup_moyenne"] = 0
for taux in taux_votes:
    for year in elections_par_annees.index.unique():
        temp = dft11[dft11["Année"]==year][taux] > elections_par_annees.loc[year][taux]
        dft11.set_value(temp.index, taux+"_sup_moyenne", temp)


In [377]:
for taux in taux_votes:
    dft11[taux+"_sup_moyenne"] = dft11.groupby("code")[taux+"_sup_moyenne"].cumsum()

### Merge data popularité

In [378]:
pop = pd.read_csv("popularite_partis.csv")
pop = pop.drop(["Unnamed: 0", "Pas Confiance"], axis=1)

In [379]:
pop.head()
pop = pop.pivot(index='Date', columns="Parti", values='Confiance')

In [380]:
pop = pop.rename(columns={"Verts":"pop_verts", "centre":"pop_centre", "droite":"pop_droite", "gauche":"pop_gauche", \
                    "xdroite":"pop_xdroite", "xgauche":"pop_xgauche"})
pop = pop.reset_index()


In [381]:
pop_prez = pd.read_csv("popularite_new.csv", delimiter=";", encoding="latin1")

In [382]:
dft11 = dft11.merge(pop, how='left', left_on=["Année"], right_on=["Date"])
dft11 = dft11.merge(pop_prez, how='left', left_on=["Année"], right_on=["Année"])

In [383]:
dft11.to_csv("la_base.csv")

### Départements dissidents 

In [384]:

elec_max = pd.melt(elections_par_annees.reset_index(), id_vars=['Année'])

idx = elec_max.groupby(['Année'])['value'].transform(max) == elec_max['value']

elec_max = elec_max[idx]

In [385]:
elec_max = elec_max.set_index("Année")

In [386]:
dft11["Dissident"] = 0
for year in elections_par_annees.index.unique():
    temp = dft11[dft11["Année"]==year][elec_max.loc[year]["variable"]] < elec_max.loc[year]["value"]
    dft11.set_value(temp.index, "Dissident", temp)

In [387]:
dft11["Dissident"] = dft11.groupby("code")["Dissident"].cumsum()

print(dft11.columns)

for item in ['20-39ans', '40-59ans', '60-74ans', '75+ans', 'Naissances domiciliées par département', \
             'Nombre total de mariages domiciliés', 'Décès domiciliés par département']:
    dft11[item] = dft11[item]*100/dft11["Total"]

Index(['département', 'code', 'Année', 'taux_Blancs et nuls', 'taux_xdroite',
       'taux_gauche', 'taux_Abstention', 'taux_xgauche', 'taux_centre',
       'taux_vert', 'taux_droite', 'depart_frontalier', 'depart_OM',
       'depart_CORSE', '0-19ans', '20-39ans', '40-59ans', '60-74ans', '75+ans',
       'Total', 'Naissances domiciliées par département',
       'Nombre total de mariages domiciliés',
       'Décès domiciliés par département', 'Q1_rate', 'Q2_rate', 'Q3_rate',
       'Q4_rate', 'var_chomage_annee', 'taux_chomage',
       'taux_centre_sup_moyenne', 'taux_droite_sup_moyenne',
       'taux_gauche_sup_moyenne', 'taux_vert_sup_moyenne',
       'taux_xdroite_sup_moyenne', 'taux_xgauche_sup_moyenne',
       'taux_Abstention_sup_moyenne', 'taux_Blancs et nuls_sup_moyenne',
       'Date', 'pop_verts', 'pop_centre', 'pop_droite', 'pop_gauche',
       'pop_xdroite', 'pop_xgauche', 'pop_premier_ministre', 'cohabitation',
       'pop_president', 'droite_au_pouvoir', 'Dissident'],
    

### Superficie / Densité

In [388]:
superficie = pd.read_csv("superficie_dept.csv")

superficie["No"] = superficie["No"].map(lambda x : "0"+str(x) if len(str(x))==1 else str(x).strip())



In [389]:
dft11 = dft11.merge(superficie, how='left', left_on=["code"], right_on=["No"])
print(dft11.columns)
dft11.drop(['No', 'Département '], axis=1, inplace=True)

Index(['département', 'code', 'Année', 'taux_Blancs et nuls', 'taux_xdroite',
       'taux_gauche', 'taux_Abstention', 'taux_xgauche', 'taux_centre',
       'taux_vert', 'taux_droite', 'depart_frontalier', 'depart_OM',
       'depart_CORSE', '0-19ans', '20-39ans', '40-59ans', '60-74ans', '75+ans',
       'Total', 'Naissances domiciliées par département',
       'Nombre total de mariages domiciliés',
       'Décès domiciliés par département', 'Q1_rate', 'Q2_rate', 'Q3_rate',
       'Q4_rate', 'var_chomage_annee', 'taux_chomage',
       'taux_centre_sup_moyenne', 'taux_droite_sup_moyenne',
       'taux_gauche_sup_moyenne', 'taux_vert_sup_moyenne',
       'taux_xdroite_sup_moyenne', 'taux_xgauche_sup_moyenne',
       'taux_Abstention_sup_moyenne', 'taux_Blancs et nuls_sup_moyenne',
       'Date', 'pop_verts', 'pop_centre', 'pop_droite', 'pop_gauche',
       'pop_xdroite', 'pop_xgauche', 'pop_premier_ministre', 'cohabitation',
       'pop_president', 'droite_au_pouvoir', 'Dissident', 'No',

In [390]:
dft11["Superficie "] = dft11["Superficie "].map(lambda x : float(x))

In [391]:
dft11["Densité"] = dft11["Total"] / dft11["Superficie "]

### Merge data présidents de départements

In [392]:
prez_depart = pd.read_csv("prez_depart.csv", delimiter=";", encoding="latin1")

In [393]:
prez_depart = pd.melt(prez_depart, id_vars=['No', "persistance_gauche", "persistance_droite", "persistance centre", "persistance_centre_droite"], value_vars=['1981', '1988', '1995', '2002', '2007', '2012', '2017'])

In [394]:
result_prez_dep = pd.concat([prez_depart, pd.get_dummies(prez_depart["value"])], axis=1)


In [395]:
result_prez_dep.drop(["value", "indep"], axis=1, inplace=True)
result_prez_dep.rename(columns={"centre":"pres_centre", "droite":"pres_droite", \
                               "gauche":"pres_gauche", "xgauche":"pres_xgauche", "variable":"Année"}, inplace=True)

In [396]:
result_prez_dep["No"] = result_prez_dep["No"].map(lambda x : "0"+str(x) if len(str(x))==1 else str(x).strip())
result_prez_dep["Année"] = result_prez_dep["Année"].map(lambda x : float(x))
dft11 = dft11.merge(result_prez_dep, how='left', left_on=["code", "Année"], right_on=["No", "Année"])
dft11.drop("No", axis=1, inplace=True)

In [397]:
dft11.to_csv("la_base.csv")

### Merge data PIP / opinion menage / depense conso menage

In [398]:
pib_impot = pd.read_csv("pib_impot_emploi_travaux.csv", delimiter=";", encoding="latin1")
pib_impot = pib_impot.groupby("année").mean().reset_index()
dft11 = dft11.merge(pib_impot, how='left', left_on="Année", right_on="année").drop(["trimestre", "année"], axis=1)

In [399]:
opinion_menage = pd.read_csv("opinion_menage.csv", delimiter=";")
opinion_menage = opinion_menage.groupby("année").mean().reset_index()
dft11 = dft11.merge(opinion_menage, how='left', left_on="Année", right_on="année").drop(["mois", "année"], axis=1)

In [400]:
dft11.to_csv("la_base.csv")

### Merge démo médecin

In [401]:
demo_medecin = pd.read_csv("demo_medecin.csv")

In [402]:
print(dft11["Année"].unique())

[ 1981.  1988.  1995.  2002.  2007.  2012.]


In [403]:
demo_medecin = pd.melt(demo_medecin, id_vars=['N0'], value_vars=['1978', '1985', '1992', '1999', '2004', '2009', '2014'])

In [404]:
demo_medecin["variable"] = demo_medecin["variable"].map(lambda x : float(x))
demo_medecin["variable"] = demo_medecin["variable"] + 3

In [405]:
demo_medecin.rename(columns={"variable":"Année", "value":"densité_médecins"}, inplace=True)

In [406]:
demo_medecin["N0"] = demo_medecin["N0"].map(lambda x : "0"+str(x) if len(str(x))==1 else str(x).strip())

In [407]:
demo_medecin["densité_médecins"] = demo_medecin["densité_médecins"].map(lambda x : float(str(x).replace(",", ".")))

In [408]:
dft11 = dft11.merge(demo_medecin, how='left', left_on=["Année", "code"], right_on=["Année", "N0"]).drop(["N0"], axis=1)

### menages logements 

In [409]:
entreprise_menage_logement = pd.read_csv("entrerise_menage_logement.csv")

In [410]:
entreprise_menage_logement = entreprise_menage_logement[["année", "consommation menages - electricite", "nombre de logement vacant"]]

In [411]:
dft11 = dft11.merge(entreprise_menage_logement, how='left', left_on=["Année"], right_on=["année"])
dft11.drop("année", axis=1, inplace=True)

In [412]:
dft11.to_csv("la_base.csv")

In [413]:
dft11.columns

Index(['département', 'code', 'Année', 'taux_Blancs et nuls', 'taux_xdroite',
       'taux_gauche', 'taux_Abstention', 'taux_xgauche', 'taux_centre',
       'taux_vert', 'taux_droite', 'depart_frontalier', 'depart_OM',
       'depart_CORSE', '0-19ans', '20-39ans', '40-59ans', '60-74ans', '75+ans',
       'Total', 'Naissances domiciliées par département',
       'Nombre total de mariages domiciliés',
       'Décès domiciliés par département', 'Q1_rate', 'Q2_rate', 'Q3_rate',
       'Q4_rate', 'var_chomage_annee', 'taux_chomage',
       'taux_centre_sup_moyenne', 'taux_droite_sup_moyenne',
       'taux_gauche_sup_moyenne', 'taux_vert_sup_moyenne',
       'taux_xdroite_sup_moyenne', 'taux_xgauche_sup_moyenne',
       'taux_Abstention_sup_moyenne', 'taux_Blancs et nuls_sup_moyenne',
       'Date', 'pop_verts', 'pop_centre', 'pop_droite', 'pop_gauche',
       'pop_xdroite', 'pop_xgauche', 'pop_premier_ministre', 'cohabitation',
       'pop_president', 'droite_au_pouvoir', 'Dissident', 'Supe